<h1><center>Stablecoin Billionaires<br> Descriptive Analysis of the Ethereum-based <br>
    Stablecoin ecosystem</center></h1>  

<h2><center> by Anton Wahrstätter<br>01.07.2020</h2></center>

<h2><center> Script to fetch Event data from Ethereum and store it as CSV</h2></center>

In [2]:
# In order to add another token, make the following additions
# 1. Add the Contract address of the token contract to the tokens
# 2. If the Event isn't already inlcuded, add it too (pay adittion to the exact expression of the code)
# 3. If you you have added a new Event manually, you have to adapt the functions handle_result 
#    and save_chunk_to_csv too

# Load libriaries

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import sha3
import time
import json
import csv
import os

# Inputs and Constants 

## Constants

In [ ]:
MAX_CALLS_PER_SECOND = 5 # max calls per second, Etherscan allows 5 at max
API = 'https://api.etherscan.io/api?module=logs&action=getLogs' # API endpoint
MAX_EVENTS = 1000 # Max events per request, Etherscan returns 1000 at max
CHUNKADJUSTER = 200 # The level, when the requested blocksize should be increased 
LINE = '-------------------------------------------------------'
LINE = LINE+LINE
print('Enter your Etherscan API Key:')
KEY = input()

## Tokens

In [ ]:
TOKEN = ''
print('Token: (lowercase)')
_TOKEN = input().lower()

    #DOESN'T emit Transfers for mint and burn but Issue and DestroyedBlackFunds
if _TOKEN == 'tether':
    TETHER = '0xdAC17F958D2ee523a2206206994597C13D831ec7'
    TOKEN = TETHER
    
    #emits Transfers for mint and burn + Mint and Burn
if _TOKEN == 'usdc':
    USDC = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
    TOKEN = USDC
    
    #emits Transfers for mint and burn + SupplyIncreased and SupplyDecreased
if _TOKEN == 'paxos':
    PAXOS = '0x8e870d67f660d95d5be530380d0ec0bd388289e1'
    TOKEN = PAXOS   

    #emits Transfers for mint and burn but no specific event
if _TOKEN == 'dai':
    DAI = '0x6B175474E89094C44Da98b954EedeAC495271d0F'
    TOKEN = DAI 

     #emits Transfers for mint and burn + Mint and Burn
if _TOKEN == 'trueusd':
    TRUEUSD = '0x0000000000085d4780B73119b644AE5ecd22b376'
    TOKEN = TRUEUSD 
    
if _TOKEN == 'oldtrueusd':
    OLDTRUEUSD = '0x8dd5fbCe2F6a956C3022bA3663759011Dd51e73E'
    TOKEN = OLDTRUEUSD 

    #emits Transfers for mint and burn + Issue and Redeem
if _TOKEN == 'husd':
    HUSD = '0xdf574c24545e5ffecb9a659c229253d4111d87e1' 
    TOKEN = HUSD 

    #emits Transfers for mint and burn + SupplyIncreased and SupplyDecreased
if _TOKEN == 'binanceusd':
    BINANCEUSD = '0x4Fabb145d64652a948d72533023f6E7A623C7C53' 
    TOKEN = BINANCEUSD 
    
       
if TOKEN:
    print('Selected Token: {}'.format(_TOKEN))
else:
    raise Exception('Token not found')

## Events

In [ ]:
print('Eventname: (lowercase)')
EVENT = ''
_EVENT = input().lower()

if _EVENT == 'transfer':
    TRANSFER = '0x' + sha3.keccak_256('Transfer(address,address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = TRANSFER
if _EVENT == 'issue':
    ISSUE = '0x' + sha3.keccak_256('Issue(uint256)'.encode('utf-8')).hexdigest()
    EVENT = ISSUE 
if _EVENT == 'redeem':
    REDEEM = '0x' + sha3.keccak_256('Redeem(uint256)'.encode('utf-8')).hexdigest()
    EVENT = REDEEM
if _EVENT == 'issuehusd':
    ISSUE = '0x' + sha3.keccak_256('Issue(address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = ISSUE 
if _EVENT == 'redeemhusd':
    REDEEM = '0x' + sha3.keccak_256('Redeem(address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = REDEEM
if _EVENT == 'destroyedblackfunds':
    DESTROYED = '0x' + sha3.keccak_256('DestroyedBlackFunds(address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = DESTROYED
if _EVENT == 'approve':
    APPROVE = '0x' + sha3.keccak_256('Approval(address,address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = APPROVE
if _EVENT == 'mint':
    MINT = '0x' + sha3.keccak_256('Mint(address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = MINT
if _EVENT == 'mintusdc':
    MINT = '0x' + sha3.keccak_256('Mint(address,address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = MINT
if _EVENT == 'burn':
    BURN = '0x' + sha3.keccak_256('Burn(address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = BURN
if _EVENT == 'supplyincreased':
    SUPPLYINCREASED = '0x' + sha3.keccak_256('SupplyIncreased(address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = SUPPLYINCREASED    
if _EVENT == 'supplydecreased':
    SUPPLYDECREASED = '0x' + sha3.keccak_256('SupplyDecreased(address,uint256)'.encode('utf-8')).hexdigest()
    EVENT = SUPPLYDECREASED 

    
if EVENT:
    print('\nSelected Event: {}'.format(_EVENT))
else:
    raise Exception('Check name of events in the Github Docs.')

## Blocks

In [ ]:
print('Startblock:')
STARTBLOCK = int(input())
print('Endblock:')
ENDBLOCK = int(input())

## File Storage Location

In [ ]:
print('Filename: (default: data/{}/{}/{}_{}.csv)'.format(_TOKEN, _EVENT, _TOKEN, _EVENT))   
FILENAME = input()
if FILENAME == '':
    FILENAME = 'data/{}/{}/{}_{}.csv'.format(_TOKEN, _EVENT, _TOKEN, _EVENT)
print('\nFilename: {}'.format(FILENAME))

## Global Variables

In [ ]:
print('Select CHUNKSIZE: (how many blocks should at the beginning be processed within one API-request - then autoregulated depending on the blocksizes)')
CHUNKSIZE = input()
if CHUNKSIZE == '':
    CHUNKSIZE = ENDBLOCK - STARTBLOCK
CHUNKSIZE = int(CHUNKSIZE)

print('Select Max. CHUNKSIZE: (depends on number of events, f.e. take max. 20,000 for Transfer Events, but much more for Issue Events)')
MAXCHUNKSSIZE = input()
if MAXCHUNKSSIZE == '':
    MAXCHUNKSSIZE = ENDBLOCK - STARTBLOCK     
MAXCHUNKSSIZE = int(MAXCHUNKSSIZE)
chunklength = np.array([CHUNKADJUSTER*1.3]*10) # intially set 30% higher than chunkadjuster 
                                               # keeps track of number of events within a response

# Helper Functions

In [ ]:
def from_hex(string):
    if str(string) == '0x':
        return 0
    return int(str(string),16)

# main function - performs request,
def request_events(block_from, block_to, event, token, bl_start):
    global CHUNK, CHUNKSIZE, counter, block_time_avg, chunklength
    ev_amount = MAX_EVENTS # set ev_amount to MAX_EVENT to start while loop
    increased = False # binary variable to tell the executing function if the requested block range increased
    inc_lock = False # binary variable to block increases - needed if very few blocks are requested
    while  ev_amount >= MAX_EVENTS:
        res_code = 400 # initally set response to 400 to enter loop
        rep = 0 # repetitions counter
        while res_code != 200:
            time.sleep(0.1)
            rep += 1
            query = '{}&fromBlock={}&toBlock={}&topic0={}&topic0_1_opr=and&address={}&apikey={}'.format(API,
                                                                                               str(block_from),
                                                                                               str(block_to),
                                                                                               event,
                                                                                               token,
                                                                                               KEY)
            res = requests.get(query) # actual request
            res_code = res.status_code # get status code of request
            if rep > 2 and res_code != 200: # if failed - wait and try again
                time.sleep(5)
                print('connection problem occured - will try again - {}'.format(rep))
                if rep == 5: # if failed 5 times, wait longer and throw exception
                    time.sleep(120)
                    raise Exception('\n\n\n\n\nConnection problem to API\n\n\n\n\n')
                    
            if json.loads(res.content.decode())["result"] == None: 
                res_code = 400 # if request successfull but no content - retry by setting res_code to 400
        result = json.loads(res.content)['result'] # get result of response
        if result == None:
            print(res.content)
            
        ev_amount = len(result) # get number of events that are included in the response
        chunklength = np.append(chunklength, [ev_amount]) # add lenght to chunklength 
        
        # -------Start of Block Range Adjuster-------
        if ev_amount >= MAX_EVENTS:
            if 0 < CHUNKSIZE <= 4:
                factor = 1
            else:
                factor = 4
            print('--- Length of results: {} ---'.format(ev_amount))
            block_to -= round((block_to - block_from)/factor)
            CHUNKSIZE = block_to - block_from
            print('--- Reducing CHUNKSIZE to: {} ---'.format(CHUNKSIZE))
            inc_lock = True
            
        if np.mean(chunklength[-10:]) < CHUNKADJUSTER and CHUNKSIZE*1.125 < MAXCHUNKSSIZE and inc_lock == False:
            _block_to = block_to
            if CHUNKSIZE <= 4:               
                factor = 1
            else:
                factor = 8
            print('+++ Length of results: {} +++'.format(ev_amount))
            block_to += round((block_to - block_from)/factor)
            CHUNKSIZE = block_to - block_from
            print('+++ Increasing CHUNKSIZE to: {} +++'.format(CHUNKSIZE))
            increased = round((_block_to - block_from)/factor)
        # -------End of Block Range Adjuster-------
        
    counter += 1 # counter for printing a headline      
    handle_result(result) # fetch data from response and add it to the chunk
    bl_delta = datetime.now().timestamp() - bl_start  # request duration
    block_time_avg = np.append(block_time_avg, [bl_delta]) # average time to request a block
    if len(block_time_avg) > 5: # keep track of average of the last 5 only
        block_time_avg = block_time_avg[1:]
    if counter % 1 == 0: # if allways, can easily be adapted to only print every 2nd time
        try: # print in the case of an increased blockrange
            print_logs(True, counter, (block_from, block_to-increased), CHUNK[-1][0], ev_amount, np.mean(block_time_avg))
            block_time_avg = np.array([])
        except:
            print_logs(True, counter, (block_from, block_to), '-', ev_amount, np.mean(block_time_avg))
            block_time_avg = np.array([])

    return increased

# takes the http request and adds it to the CHUNK
def handle_result(result):
    global CHUNK
    for e in result:
        value = from_hex(e['data'])
        block_number = from_hex(e['blockNumber'])
        time_stamp = from_hex(e['timeStamp'])
        tx_hash = e['transactionHash']
        tx_index = from_hex(e['transactionIndex'])
        gas_price = from_hex(e['gasPrice'])
        gas_used = from_hex(e['gasUsed'])

        if _EVENT in ['transfer', 'approve']:
            tx_from, tx_to = '0x' + e['topics'][1][-40:], '0x' + e['topics'][2][-40:]
            CHUNK.append([time_stamp, block_number, tx_index, tx_hash, tx_from, tx_to, value, gas_price, gas_used])

        if _EVENT in ['issue', 'redeem']:
            CHUNK.append([time_stamp, block_number, tx_index, tx_hash, value, gas_price, gas_used])

        if _EVENT in ['destroyedblackfunds']:
            address = '0x' + e['data'][26:66]
            funds = from_hex('0x' + e['data'][-20:])
            CHUNK.append([time_stamp, block_number, tx_index, tx_hash, address, funds, gas_price, gas_used])
        
        if _EVENT in ['mint', 'burn', 'supplyincreased', 'supplydecreased', 'issuehusd', 'redeemhusd']:
            address = '0x' + e['topics'][1][-40:]
            CHUNK.append([time_stamp, block_number, tx_index, tx_hash, address, value, gas_price, gas_used])
            
        if _EVENT in ['mintusdc']:
            address = '0x' + e['topics'][2][-40:]
            CHUNK.append([time_stamp, block_number, tx_index, tx_hash, address, value, gas_price, gas_used])
            
    return []


# print logs
def print_logs(arg, *args):
    counter, block_number, bl_time_stamp, ev_amount, block_time_avg = args
    bl_f, bl_t = block_number
    if arg:
        if counter == 1:
            print('{:<8}{:<25}{:<20}{:<20}{:<10}{:<14}{}'.format('Index',
                                                                 'UTC Timestamp',
                                                                 'Block Nr.',
                                                                 'Block Timestamp',
                                                                 'Events',
                                                                 'Requests/s',
                                                                 'Blocks/s'))
            print(LINE)
        
        
        print('{:<6}{:<25}{:<20}{:<25}{:<11}{:<11.2f}{:d}'.format(counter,
                                                                  str(datetime.utcnow())[0:-7],
                                                                  str(bl_f) + '/' + str(bl_t),
                                                                  bl_time_stamp if bl_time_stamp == '-' else datetime.utcfromtimestamp(bl_time_stamp).strftime('%Y-%m-%d %H:%M:%S'),
                                                                  ev_amount,
                                                                  1/block_time_avg,
                                                                  int((bl_t-bl_f)/block_time_avg)
                                                                  ))

# save current CHUNK to CSV and reset CHUNK
def save_chunk_to_csv():
    global CHUNK
    if not os.path.exists(FILENAME):
        if _EVENT in ['transfer', 'approve']:
            df = pd.DataFrame(columns=['timestamp','blocknumber','txindex','txhash','txfrom','txto', 'txvalue', 'gas_price', 'gas_used'])
        if _EVENT in ['issue', 'redeem']:
            df = pd.DataFrame(columns=['timestamp','blocknumber','txindex','txhash', 'txvalue', 'gas_price', 'gas_used'])
        if _EVENT in ['destroyedblackfunds', 'mint', 'mintusdc', 'burn', 'supplyincreased', 'supplydecreased', 'issuehusd', 'redeemhusd']:
            df = pd.DataFrame(columns=['timestamp','blocknumber','txindex','txhash', 'address', 'txvalue', 'gas_price', 'gas_used'])
        
        df.to_csv(FILENAME, index = False)
    with open(FILENAME, 'a') as f:
            writer = csv.writer(f)
            writer.writerows(e for e in CHUNK)  
    CHUNK = []

print('Selected Chunksize/Max-Chunksize: {}/{}'.format(CHUNKSIZE, MAXCHUNKSSIZE))

# Run Script

In [ ]:
counter = 0
CHUNK = []
block_time_avg = np.array([])
def mine_events():
    global STARTBLOCK, CHUNKSIZE
    print('\nSTART MINING: {}'.format(TOKEN))
    print('UTC TIME: {}'.format(str(datetime.utcnow())[0:-7]))
    print('\nFROM BLOCK {}'.format(STARTBLOCK))
    print('TO BLOCK {}\n'.format(ENDBLOCK))
    _BLOCK = STARTBLOCK
    
    while _BLOCK <= ENDBLOCK: 
        if CHUNKSIZE == 0:
            CHUNKSIZE = 1
        bl_start = datetime.now().timestamp()
        step_t_i = _BLOCK
        step_t_j = _BLOCK + CHUNKSIZE 
        if step_t_j > ENDBLOCK:
            step_t_j = ENDBLOCK
        delta = request_events(step_t_i, step_t_j, EVENT, TOKEN, bl_start)
        save_chunk_to_csv()
        _BLOCK = _BLOCK + CHUNKSIZE + 1 - delta
        print('### Current Filesize: {} MB ###'.format(os.path.getsize(FILENAME)/10**6)) if _BLOCK % 50 == 0 and os.path.exists(FILENAME) else False
        
    print('\nFINISHED MINING FROM BLOCK NUMBER {} TO BLOCK NUMBER {}'.format(STARTBLOCK, ENDBLOCK))
    print('\nFILE SAVED UNDER {}'.format(FILENAME))
    

mine_events()  